In [23]:
import tkinter as tk
import pandas as pd
import pandastable as pt
features = ['Name of Judge', 'Remark']
df = pd.DataFrame('N/A', index=range(1, 7 + 1), columns=features)
df

,Name of Judge,Remark
1,N/A,N/A
2,N/A,N/A
3,N/A,N/A
4,N/A,N/A
5,N/A,N/A
6,N/A,N/A
7,N/A,N/A


In [32]:
root = tk.Tk()
table = pt.Table(root, dataframe= df, showtoolbar=True, showstatusbar=True)
table.show()
table.grid(row = 1, column = 1)
root.mainloop()

AttributeError: 'NoneType' object has no attribute 'bind_all'

In [8]:
# T001S01.csv

THRESHOLD_REF = {3:13, 5:23, 7:33}

class Process:
	__slots__ = 'id', 'judges', 'stages', 'iterator', 'immutable', 'observers'

	def __init__(self, id, judges, stages, immutable = False):
		self.id = id
		self.judges = judges
		self.stages = stages
		self.iterator = self.stage_count() - 1
		self.immutable = immutable
		self.observers = []

	def move(self, stage_no):
		try:
			stage = self.stages[stage_no - 1]
			self.iterator = stage_no - 1
		except IndexError:
			stage = None
		finally:
			return stage

	def skip(to):
		self.save()

		while self.iterator < to:
			self.stages.append(None)
			self.iterator += 1

		self.iterator -= 1
		self.append_stage()
		return self.cur_stage()



	def forward(self, next):
		if self.immutable:
			return self.move(self.cur_stage() + 1)

		self.save()
		self.append_stage()		
		self.update_ob()

		return self.cur_stage()

	def append_stage(self):
		self.stages.append(Stage(self.stages[self.iterator].filter_by_total(self.threshold())))
		self.iterator += 1

	def save(self):
		self.stages[self.iterator].sort()
		self.stages[self.iterator].save(self.cur_path())

#	  5   7  3
# 1 - 3   4  2
# 2 - 23 33  
# 3 - 25 35
# 4 - 27 37
# Finalize
	def threshold(self):
		stage = self.cur_stage_no()
		n_judges = len(self.judges)

		if stage == 1:
			t = int(n_judges / 2) + 1
		else:
			t = THRESHOLD_REF[n_judges]
			idx = 2
			while idx < stage:
				t += 2
				idx += 1
		return threshold

	def cur_mode(self):
		return 0 if self.cur_stage_no() == 1 else 1

	def cur_path(self):
		return 'resource/history/T' + str(self.id) + 'S' + str(self.cur_stage_no()) + '.csv'

	def init_candidates(self):
		return self.stages[0].candiates()

	def stage_count(self):
		return len(self.stages)

	def cur_candidate(self):
		return self.stages[self.iterator].cur_candidate()

	def cur_stage_no(self):
		return self.iterator + 1

	def cur_stage(self):
		return self.stages[self.iterator]

	def next_candidate(self):
		candidate = self.stages[self.iterator].next_candidate()
		if candidate != -1:
			self.update_ob()
		return candidate

	def add_ob(self, observer):
		self.observers.append(observer)

	def update_ob(self):
		for ob in self.observers:
			ob.update(self.cur_stage_no(), self.cur_candidate())

class Stage:
	__slots__ = 'data', 'iterator', 'immutable'

	def __init__(self, data, immutable = False):
		self.data = data
		self.iterator = None if immutable else self.candiates()
		self.immutable = immutable

	# The caller need to ask for forgiveness
	# IndexError
	def __setitem__(self, key, value):
		self.data.loc[self.iterator][key] += value
		self.data.loc[self.iterator]['Total'] += value

	def dataframe(self):
		return self.data

	# TODO: file system structure
	def save(self, path):
		self.data.to_csv(path, sep = ',', encoding = 'utf-8')

	def candiates(self):
		return len(self.data.loc[:])

	def sort(self):
		self.data.sort_values(['Total', 'Recap'], ascending = [False, False], inplace = True)

	def filter_by_total(self, threshold):
		logic = self.data.loc[:]['Total'] > (threshold - 1)
		# TODO: set all to zeros
		return self.data[:][logic]

	def cur_candidate(self):
		return self.iterator

	def next_candidate(self):
		if self.iterator > 1:
			self.iterator -= 1
			return self.iterator
		else:
			return -1

class ProcessBuilder:
	__slots__ = 'id', 'stk', 'cur', 'judges'

	def __init__(self, id, data, judges):
		self.id = id
		self.stk = [None for i in range(0, 15)]
		self.cur = 1
		self.stk[0] = Stage(data)
		self.judges = judges

	def __enter__(self):
		return self

	def __exit__(self, exc_type, exc_value, traceback):
		self.stk = None
		self.cur = 0

	def add(self, data):
		self.stk[self.cur] = Stage(data)
		self.cur += 1

	def build(self):
		return Process(self.id, self.judges, self.stk[0: self.cur])

	def build_immutable(self):
		return Process(self.id, self.judges, self.stk[0: self.cur], immutable = True)

def process_new(id, data, judges):
	with ProcessBuilder(id, data, judges) as builder:
		process = builder.build()
	return process

def process_old(urls):
	pass


In [9]:
# Builder Test
n_judges = 5
judges = [str(i) for i in range(1, n_judges + 1)]
n = 10
fields = judges + ['Total', 'Recap']
process = process_new(pd.DataFrame(0, index = range(1, n + 1), columns = fields), judges)

In [10]:
for idx in range(0, 5):
    process.cur_stage()['1'] = 1

In [29]:
# Process -> forward : passed
process.forward(1)

In [34]:
# Process -> move : Passed
process.move(2)

In [31]:
#process.forward(5)
judges

['1', '2', '3', '4', '5']

In [45]:
process.cur_stage()['1'] = 1
process.cur_stage().dataframe()

,1,2,3,4,5,Total,Recap
1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0
9,1,0,0,0,0,1,0
10,0,0,0,0,0,0,0


In [66]:
import sys
sys.path.append('../')

import config.key_config as config

ON = 1
OFF = 0

class Key:
	__slots__ = 'key', 'states', 'state'

	def __init__(self, key, on, off):
		self.key = key
		self.states = [on, off]
		self.state = ON

	def switch(self, state = OFF):
		self.state = state

	def check(self, key):
		return key.lower() == self.key

	def __call__(self, value):
		self.states[self.state].handle(value)

class KeyMap:
	__slots__ = 'keys', 'scheme', 'enabled'

	def __init__(self, keys, scheme = 0, enabled = True):
		self.keys = keys
		self.scheme = scheme
		self.enabled = enabled

	def count(self):
		return len(self.keys) / 3

	def set_scheme(scheme = 1):
		self.scheme = scheme

	def disable(self, enabled = False):
		self.enabled = enabled

	def __call__(self, key):
		if not self.enabled:
			pass

		for group in self.keys.keys():
			idx = 0
			n = len(self.keys[group])
			while idx < n:
				if self.keys[group][idx].check(key):
					print(config.SCHEME[self.scheme])
				idx += 1

def keymap(judges):
	k_map = {}
	keys = config.get_keys(judges)
	for judge, k_list in zip(keys.keys(), keys.values()):
		k_map[judge] = []
		idx = 0
		while idx < 3:
			k_map[judge].append(Key(k_list[idx], None, None))
			idx += 1
	return KeyMap(k_map)


In [67]:
k_map = keymap(judges)
k_map('w')

(0, 1, 1)


In [33]:
counts = df.loc[[1,2, 3]].groupby(['Total', 'TB']).size()
counts = counts[counts > 1]
group = []
for entry in counts.index:
	logic = (df['Total'] == entry[0]) & (df['TB'] == entry[1])
	group.extend(df.index[logic].tolist())
group

[1, 2]